In [ ]:
!gshell init

In [ ]:
!gshell info --with-id 17axH-dYh-Wss_ec92ugSl8AmSuZS-xn9

In [ ]:
import signal
from pathlib import Path
import wget

parser_version = '1.6.4'
url = f'https://github.com/nemoware/document-parser/releases/download/{parser_version}/document-parser-{parser_version}.jar'
if not Path(f'document-parser-{parser_version}.jar').is_file():
    wget.download(url)

In [ ]:
import glob
import zipfile

with zipfile.ZipFile(f'./{glob.glob("ДД по практикам*.zip")[0]}',
                     'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
key_value = ['о нижеследующем:', 'нижеследующем:', 'о нижеследующем', 'нижеследующем']
for x in key_value[5:]:
    print(123)


In [ ]:
parser_version = '1.6.4'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "ДД по практикам\Практика недропользования и экологии\1. ДС N 8_испр.docx"

In [ ]:
import subprocess

s = [
    "gshell",
    "init"
]

subprocess.run(s, stdin=subprocess.PIPE, stdout=subprocess.PIPE)

## Start

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time
import re
import numpy as np

index = 1
number_of_docs = 0
parser_version = '1.6.4'
root = "ДД по практикам"

with_neuron = False

tokenizer_path = "sberbank-ai/ruRoberta-large"
path_to_model = "./doc-classification/"

labels = ['Практика коммерческой логистики',
          'Практика недропользования и экологии',
          'Практика поддержки региональных, розничных продаж и клиентского сервиса',
          'Практика правового сопровождения закупок МТР и услуг общего профиля',
          'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
          'Практика правового сопровождения операционной деятельности БРД',
          'Практика правового сопровождения переработки и инфраструктуры',
          'Практика правовой поддержки брендов',
          'Практика правовой поддержки использования и коммерциализации ИС',
          'Практика правовой поддержки создания и приобретения ИС',
          'Практика промышленной безопасности и охраны труда',
          'Практика финансового и конкурентного права',
          'Практика экспорта, оптовых продаж и сбыта бизнес-единиц (БЕ)']

arrOfAllDocs = []
result = []
result_of_fail = []
result_of_possible = []
result_of_possible2 = []

s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

for root, dirnames, filenames in os.walk('ДД по практикам'):
    if len(root.split('\\')) == 1: continue
    flag = False
    for i in root.split('\\'):
        if str(i).startswith('Исключена'):
            flag = True
            break
    if flag: continue
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))

print(
    "Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс"
)
java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                   stdout=subprocess.PIPE, encoding="utf-8")


def find_let(document, documentType=None):
    key_value = ['о нижеследующем:', 'нижеследующем:', 'о нижеследующем', 'нижеследующем']
    if documentType == 'SUPPLEMENTARY_AGREEMENT':
        key_value.append('в следующей редакции:')
        key_value.append('заключили настоящее Дополнительное соглашение к Договору.')
        key_value.append('редакции:')
    result = ""
    for i, p in enumerate(document['paragraphs']):
        if any(f.lower() in p['paragraphBody']['text'].lower() or f.lower() in
               p['paragraphHeader']['text'].lower() for f in
               key_value[:4]) or any(f.lower() in p['paragraphBody']['text'].lower() or f.lower() in
                                     p['paragraphHeader']['text'].lower() for f in
                                     key_value[5:]):
            text = ""
            for x in key_value[:4]:
                if x.lower() in p['paragraphBody']['text'].lower():
                    text += p['paragraphBody']['text'].split(x)[1]
                    break
                if x.lower() in p['paragraphHeader']['text'].lower():
                    text += p['paragraphBody']['text']
                    break

            if text == "":
                for x in key_value[5:]:
                    if x.lower() in p['paragraphBody']['text'].lower():
                        text += p['paragraphBody']['text'].split(x)[1]
                        break
                    if x.lower() in p['paragraphHeader']['text'].lower():
                        text += p['paragraphBody']['text']
                        break

            if text == "": continue

            text += "".join(
                str(x['paragraphBody']['text']) for x in document['paragraphs'][i + 1:i + 4])

            textHeader = p['paragraphHeader']['text'] + "\n".join(
                str(x['paragraphHeader']['text']) for x in document['paragraphs'][i:i + 4])

            d = i + 4
            while len(text.split()) < 300 and d < len(document['paragraphs']):
                text += document['paragraphs'][d]['paragraphBody']['text']
                d += 1

            result = {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": p['paragraphBody']['offset'],
                "text": text,
                "length": len(text),
                "offsetHeader": p['paragraphHeader']['offset'],
                "textHeader": textHeader,
                "lengthHeader": len(textHeader),
                "isTrue": is_true(text, docs)
            }
            break
    return result


def find_currency_header(paragraph, keys):
    paragraph['paragraphHeader']['text'] = re.sub(' +', ' ', paragraph['paragraphHeader']['text'])
    header_text_in_low_reg = paragraph['paragraphHeader']['text'].lower()
    basic_text_in_low_reg = paragraph['paragraphBody']['text'].lower()
    if paragraph['paragraphBody']['length'] < 20:
        return False
    if len(basic_text_in_low_reg.split()) < 15:
        return False
    if re.search("(:)\s*$", basic_text_in_low_reg):
        return False

    for key in keys:
        if key.lower() in header_text_in_low_reg:
            if 'Статья'.lower() in key.lower() and any(
                    x.lower() in header_text_in_low_reg for x in
                    ['Термины и определения', 'Термин', 'определения']):
                return False

            return True

    return False


def get_tokens(text):
    result = tokenizer(text, truncation=True, max_length=512)
    return result


def predicate_result(text):
    tokens = get_tokens(text)
    predictions = model.predict([tokens['input_ids']])['logits']
    predictions = tf.nn.softmax(predictions, name=None)[0].numpy()
    print(predictions)
    print(sum(predictions))

    return predictions


def is_true(text, path):
    if with_neuron: return False
    result = predicate_result(text)
    argmax = np.argmax(result, axis=0)
    required_label = labels[argmax]
    if required_label in path:
        return True

    return False


time.sleep(2)
i = 1
while True:
    time.sleep(0.1)
    output_log_spring = java_subprocess.stdout.readline()
    sys.stdout.write("\rПроверка соединения #%i" % i)
    sys.stdout.flush()
    i += 1
    if output_log_spring.find("Started DocumentParserService") != -1:
        print("\nГотово")
        java_subprocess.stdout.close()
        break

print("Запустился успешно")
print("Общее количество документов =", len(arrOfAllDocs))

tokenizer = None
model = None

if with_neuron:
    tokenizer = AutoTokenizer.from_pretrained(str(tokenizer_path))
    model = TFAutoModelForSequenceClassification.from_pretrained(
        str(path_to_model), num_labels=len(labels), from_pt=False
    )

for docs in arrOfAllDocs:
    try:
        file = open(docs, 'rb')
        encoded_string = base64.b64encode(file.read())
        encoded_string = str(encoded_string)[2:-1]
    except Exception as e:
        # print(f"\nОшибка в файле {docs}")
        # print(f"при конвертации в base64, исключение = {e.msg}")
        # print("=" * 200)
        continue

    response = requests.post(
        "http://localhost:8083/document-parser",
        data=json.dumps({
            "base64Content": encoded_string,
            "documentFileType": docs.split(".")[-1].upper()
        }),
        headers=headers
    )
    resArr = []
    try:
        resArr = response.json()['documents']
    except Exception as e:
        # print(f"\nОшибка в файле {docs}")
        # print(f"Ответ от парсера {response.json()}")
        # print(f"Исключение = {e}")
        # print("=" * 200)
        continue

    sys.stdout.write(f"\rПроверка документа под номером {index} и ИД {len(result)}")
    sys.stdout.flush()
    index += 1

    document = []
    if resArr:
        document = resArr[0]
    else:
        continue

    if document['documentType'] == "CONTRACT" or document['documentType'] == "AGREEMENT":
        for ind, par in enumerate(document['paragraphs']):
            document['paragraphs'][ind]['paragraphBody']['text'] = re.sub('_+', '',
                                                                          par['paragraphBody'][
                                                                              'text'])

        keys = ['Общие ', 'Общие сведения', 'Общие положение', 'Общие условия', 'Статья 1']

        if document['documentType'] == "CONTRACT":
            sup_keys = ['предмет договра', 'предмет договора', 'Предмет контракта', 'Предмет догов',
                        'Предмет и общие условия договора']
            keys = sup_keys + keys
        if document['documentType'] == "AGREEMENT":
            keys.append('Предмет соглашения')

        flag = False
        for i, p in enumerate(document['paragraphs']):
            if find_currency_header(paragraph=p, keys=keys):
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length'],
                        "isTrue": is_true(p['paragraphBody']['text'], docs)
                    })
                flag = True
                break
        if flag: continue

        all_text = "".join(
            str('\n' + x['paragraphHeader']['text'] + '\n' + x['paragraphBody']['text']) for x in
            document['paragraphs'])
        all_text = re.sub(' +', ' ', all_text)
        text_from = ""

        for key in keys:
            if key.lower() in all_text.lower():
                array_of_text = re.split(f"(?i)({key})", all_text)
                end_text = 0
                try:
                    last_symbol = re.search("\s\d[ .]\d?[\s|\u00A0|.\s]*$", array_of_text[0])
                    if last_symbol:
                        end_text = int(last_symbol.group().replace(" ", "").split(".")[0])
                    else:
                        text_from = " ".join(array_of_text[2].split()[:300])
                        # print("end_text =",end_text)
                except ValueError as ex:
                    print(f"cannot converted str to int")
                    text_from = " ".join(array_of_text[2].split()[:300])
                    break

                if end_text:
                    end_text += 1
                    print("\nEnd = ", end_text)
                    text_from = re.split(f"\s({end_text})[. ]", array_of_text[2])[0]
                    break
        if text_from != "":
            result.append({
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": text_from,
                "length": len(text_from),
                "offsetHeader": p['paragraphHeader']['offset'],
                "textHeader": p['paragraphHeader']['text'],
                "lengthHeader": p['paragraphHeader']['length'],
                "isTrue": is_true(p['paragraphBody']['text'], docs)
            })
            continue

        obj = find_let(document)
        if obj != "":
            result.append(obj)
            flag = True

        if flag: continue
        result_of_fail.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "\n+++++++++++++\n".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })
    elif document['documentType'] == "SUPPLEMENTARY_AGREEMENT":
        for ind, par in enumerate(document['paragraphs']):
            document['paragraphs'][ind]['paragraphBody']['text'] = re.sub('_+', '',
                                                                          par['paragraphBody'][
                                                                              'text'])
        flag = False
        for i, p in enumerate(document['paragraphs']):
            if any(f.lower() in p['paragraphHeader']['text'].lower() for f in
                   ['Статья 1']) and p['paragraphBody'][
                'length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length'],
                        "isTrue": is_true(p['paragraphBody']['text'], docs)
                    })
                flag = True
                break
        if flag: continue
        obj = find_let(document, document['documentType'])
        if obj != "":
            result.append(obj)
            flag = True

        if flag: continue
        #document['paragraphs'][0]['paragraphHeader']['text']
        result.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs']),
                "isTrue": is_true("".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']), docs)
            })

#Смерть java процессу!
if platform.system() == 'Windows':
    subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
elif platform.system() == 'Linux':
    os.kill(java_subprocess.pid, signal.SIGTERM)
else:
    print('Не известная платформа, убейте в ручную процесс java')
# os.killpg(os.getpgid(java_subprocess.pid), signal.SIGTERM)

writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

df = pd.DataFrame(result)
df.to_excel(writer, 'good', engine='xlsxwriter')
sheets_good = writer.sheets['good']
sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_fail)
df.to_excel(writer, 'SO SO', engine='xlsxwriter')
sheets_bad = writer.sheets['SO SO']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

# df = pd.DataFrame(result_of_possible)
# df.to_excel(writer, 'SO SO SO', engine='xlsxwriter')
# sheets_bad = writer.sheets['SO SO SO']
# sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])
#
# df = pd.DataFrame(result_of_possible2)
# df.to_excel(writer, 'О ниже', engine='xlsxwriter')
# sheets_bad = writer.sheets['О ниже']
# sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

writer.save()
print("\nФайл создан")